# This code is used for counting N-Grams other than single words

## Do not count singles words with this code as this code is much slower for single words

In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
from pyspark.ml.feature import NGram

In [3]:
import pymongo

In [4]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient['newsTemp']   # Enter your database name
mycol = mydb['articles']

In [5]:
records = mycol.find()

In [7]:
uniqueWordCount = {}
wordCount = {}

In [8]:
n_val = 2  # Value of N for N-grams

In [9]:
i = 1
for row in records:
    content = row['content']
    content = content.split()
    content = list(filter(lambda x: x.isalnum(), content))
    content = list(map(lambda x: x.lower(), content))
    if (content is None) or (len(content) == 0):
        content = ['']
        
    dataFrame = spark.createDataFrame([
        (0, content)
    ], ['docID', 'words'])
    
    ngram = NGram(n=n_val, inputCol="words", outputCol="ngrams")
    ngramDataFrame = ngram.transform(dataFrame)
    allNgrams = ngramDataFrame.collect()[0]['ngrams']
    
    for word in allNgrams:
        if word == '':
            continue
        temp1 = wordCount.pop(word, False)
        if temp1 is False:
            wordCount[word] = {row['date']: 1}
        else:
            temp2 = temp1.pop(row['date'], False)
            if temp2 is False:
                temp1[row['date']] = 1
            else:
                temp1[row['date']] = temp2 + 1
            wordCount[word] = temp1
            
        temp = uniqueWordCount.pop(word, False)
        if temp is False:
            uniqueWordCount[word] = 1
        else:
            uniqueWordCount[word] = temp + 1
        
    # Just to check progress
    i += 1
    if i % 1000 == 0:
        print "Completed " + str(i)
    

Completed 1000
Completed 2000
Completed 3000
Completed 4000


KeyboardInterrupt: 

In [ ]:
res = [None] * len(wordCount.keys())

In [ ]:
i = 0
for word in wordCount.keys():
    tempObj = {
        "word": word,
        "countByDate": wordCount[word]
    }
    res[i] = tempObj
    i += 1

In [ ]:
newDb = myclient['ngramCounts']
newCol = newDb['allNgrams']
x = newCol.insert_many(res)